In [22]:
from pyspark import SparkContext

In [23]:
sc = SparkContext(master="local", appName="T&C")

In [24]:
rdd1 = sc.parallelize([1,2,3])
type(rdd1)

pyspark.rdd.RDD

In [25]:
rdd1.collect()

[1, 2, 3]

In [26]:
sc

<SparkContext master=local appName=T&C>

In [27]:
!ls files/

deporte.csv           deportistaError.csv   modelo_relacional.jpg
deportista.csv        evento.csv            paises.csv
deportista2.csv       juegos.csv            resultados.csv


In [28]:
PATH = 'files/'
equiposOlimpicosRDD = sc.textFile(PATH + 'paises.csv') \
    .map(lambda line : line.split(","))

In [29]:
equiposOlimpicosRDD.take(5)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG']]

In [19]:
sc.stop()